# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [3]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [4]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
from emergency import miniproject6

In [5]:
%reload_ext autoreload
from emergency import miniproject6

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

###
 (0) class 선언

In [5]:
# class 선언
project_class = miniproject6()

응급실 데이터 필요합니다


### (1) open ai key 등록

In [6]:
project_class.load_key_file(api_path='/content/drive/MyDrive/project6_2/api_key.txt')

### (2) audio to text

In [7]:
audio_text = project_class.audio_to_text(path+"/audio/audio2.mp3")
print(audio_text)

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?



### (3) text summary

In [8]:
summary = eval(project_class.text_to_summary(audio_text))["summary"]
print(summary)

고열(38도) 및 두통, 어지러움 등 증상이 있어 응급실 방문이 필요합니다.


### (4) 응급실 등급분류

In [14]:
# 모델 불러오기
# project_class.load_bert_model(path+"fine_tuned_bert_added_10/")
# project_class.load_tokenizer(path+"fine_tuned_bert_added_10/")

# prediction 테스트
emergency = project_class.model_prediction(summary)[0] + 1

if emergency < 4:
  print("응급실을 가셔야 합니다")
else:
  print("응급실 안가도 될 것 같습니다")

응급실을 가셔야 합니다


### (5) 응급실추천

In [16]:
project_class.load_maps_key_file(path=path+'map_key.txt')

if emergency < 4:
  print("응급실 ㄱㄱ")
  project_class.recommend_hospital3(37.358827, 127.114919) #KT 본사 위치
else:
  print("응급실 안가도 될 것 같습니다")

응급실 ㄱㄱ
주변 응급실 총 개수: 90
병원 이름: 분당서울대학교병원 // 주소: 경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원) // 전화번호: 031-787-2114 // 응급실번호: 031-787-3119 // 거리: 2.773km
병원 이름: 대진의료재단분당제생병원 // 주소: 경기도 성남시 분당구 서현로180번길 20 (서현동) // 전화번호: 031-779-0114 // 응급실번호: 031-779-0119 // 거리: 4.372km
병원 이름: 국군수도병원 // 주소: 경기도 성남시 분당구 새마을로177번길 81 (율동) // 전화번호: 031-725-6075 // 응급실번호: 031-725-6119 // 거리: 6.12km


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [6]:
def project_pipeline(audio_path=None, latitude=None, longitude=None):
  # class 선언
  project_class = miniproject6()

  # 필요한 key값 불러오기
  project_class.load_key_file(api_path=api_path)
  project_class.load_maps_key_file(path=map_key_path)

  # 모델 불러오기
  project_class.load_bert_model(model_path)
  project_class.load_tokenizer(model_path)

  # whisper 테스트
  if audio_path:
    try:
      text = project_class.audio_to_text(audio_path)
    except:
      raise ValueError("분석할 오디오 파일이 존재하지 않습니다!")
  else:
    raise ValueError("분석할 대상이 없어요!")
  print("입력된 텍스트:", text)

  summary = project_class.text_to_summary(text)
  print("요약된 텍스트:", summary)

  # prediction 테스트
  emergency = project_class.model_prediction(summary)[0] + 1
  print(f"KTAS {emergency}단계 입니다.")

  if emergency < 4:
    print("근처의 응급실을 확인중입니다.")
    if latitude and longitude:
      project_class.recommend_hospital3(latitude, longitude)
    else:
      # 위치를 자체적으로 파악하여 추천
      print("위치를 파악하고 있습니다.")
  else:
    print('응급실 가실 필요가 없습니다!')

In [9]:
api_path='/content/drive/MyDrive/project6_2/api_key.txt'
map_key_path=path+'map_key.txt'
model_path=path+"fine_tuned_bert2/"

# 오디오 파일로 확인할 경우
project_pipeline(
    audio_path=path+"/audio/jihong1.m4a",
    latitude=37.358827,
    longitude=127.114919,
)

입력된 텍스트: 119죠. 지금 사람이 의식이 없습니다. 전혀 반응이 없고 깨워도 눈을 뜨지 않아요. 숨 쉬는 것도 불규칙하거나 약한 것 같습니다. 매우 위급한 상황입니다. 빨리 구급차를 보내주세요.

요약된 텍스트: 의식이 없고 반응이 없으며 눈을 뜨지 않고 숨도 약하고 불규칙한 상태인 위급한 환자.
KTAS 1단계 입니다.
근처의 응급실을 확인중입니다.
주변 응급실 총 개수: 90
병원 이름: 대진의료재단분당제생병원 // 주소: 경기도 성남시 분당구 서현로180번길 20 (서현동) // 전화번호: 031-779-0114 // 응급실번호: 031-779-0119 // 거리: 5.013km // 소요시간: 0시간 13분
병원 이름: 국군수도병원 // 주소: 경기도 성남시 분당구 새마을로177번길 81 (율동) // 전화번호: 031-725-6075 // 응급실번호: 031-725-6119 // 거리: 6.12km // 소요시간: 0시간 15분
병원 이름: 차의과학대학교분당차병원 // 주소: 경기도 성남시 분당구 야탑로 59 (야탑동) // 전화번호: 031-780-5000 // 응급실번호: 031-780-5840 // 거리: 7.039km // 소요시간: 0시간 18분


In [11]:
api_path='/content/drive/MyDrive/project6_2/api_key.txt'
map_key_path=path+'map_key.txt'
model_path=path+"fine_tuned_bert2/"

# 오디오 파일로 확인할 경우
project_pipeline(
    audio_path=path+"/audio/jihong2.m4a",
    latitude=37.358827,
    longitude=127.114919,
)

입력된 텍스트: 119죠. 지금 옆에 한 사람이 계획적으로 자살을 시도한 것 같습니다. 의식은 있지만 매우 불안정하고 위험한 상태입니다. 지금 상황이 매우 심각해서 신속한 도움과 응급 조치가 필요합니다. 빠르게 와주실 수 있나요?

요약된 텍스트: 계획적으로 자살을 시도한 환자로 매우 불안정하고 위험한 상태인 환자입니다.
KTAS 2단계 입니다.
근처의 응급실을 확인중입니다.
주변 응급실 총 개수: 90
병원 이름: 대진의료재단분당제생병원 // 주소: 경기도 성남시 분당구 서현로180번길 20 (서현동) // 전화번호: 031-779-0114 // 응급실번호: 031-779-0119 // 거리: 5.013km // 소요시간: 0시간 13분
병원 이름: 국군수도병원 // 주소: 경기도 성남시 분당구 새마을로177번길 81 (율동) // 전화번호: 031-725-6075 // 응급실번호: 031-725-6119 // 거리: 6.12km // 소요시간: 0시간 15분
병원 이름: 차의과학대학교분당차병원 // 주소: 경기도 성남시 분당구 야탑로 59 (야탑동) // 전화번호: 031-780-5000 // 응급실번호: 031-780-5840 // 거리: 7.039km // 소요시간: 0시간 18분


In [17]:
api_path='/content/drive/MyDrive/project6_2/api_key.txt'
map_key_path=path+'map_key.txt'
model_path=path+"fine_tuned_bert2/"

# 오디오 파일로 확인할 경우
project_pipeline(
    audio_path=path+"/audio/jihong3.m4a",
    latitude=37.358827,
    longitude=127.114919,
)

입력된 텍스트: 안녕하세요. 119죠? 지금 숨을 쉬는 게 약간 힘들고 호흡이 조금 곤란한 상태예요. 크게 심각하지는 않지만 숨이 차고 답답한 느낌이 있습니다. 병원에 가야 할지 조언을 부탁드려요.

요약된 텍스트: 호흡이 조금 힘들고 곤란하며 숨이 차고 답답한 환자입니다.
KTAS 3단계 입니다.
근처의 응급실을 확인중입니다.
주변 응급실 총 개수: 90
병원 이름: 대진의료재단분당제생병원 // 주소: 경기도 성남시 분당구 서현로180번길 20 (서현동) // 전화번호: 031-779-0114 // 응급실번호: 031-779-0119 // 거리: 5.013km // 소요시간: 0시간 13분
병원 이름: 국군수도병원 // 주소: 경기도 성남시 분당구 새마을로177번길 81 (율동) // 전화번호: 031-725-6075 // 응급실번호: 031-725-6119 // 거리: 6.12km // 소요시간: 0시간 15분
병원 이름: 차의과학대학교분당차병원 // 주소: 경기도 성남시 분당구 야탑로 59 (야탑동) // 전화번호: 031-780-5000 // 응급실번호: 031-780-5840 // 거리: 7.68km // 소요시간: 0시간 17분


In [16]:
api_path='/content/drive/MyDrive/project6_2/api_key.txt'
map_key_path=path+'map_key.txt'
model_path=path+"fine_tuned_bert2/"

# 오디오 파일로 확인할 경우
project_pipeline(
    audio_path=path+"/audio/jihong5.m4a",
    latitude=37.358827,
    longitude=127.114919,
)

입력된 텍스트: 안녕하세요. 119죠? 제가 만성적인 환각 증상이 있어서 지금 약간 불안하고 초조한 상태입니다. 증상이 심하지는 않지만 마음이 불안정해지고 조금 불편한 느낌이 있습니다. 응급상황은 아닌 것 같은데 조언이나 도움이 필요할 것 같습니다.

요약된 텍스트: 만성적인 환각 증상을 가진 불안하고 초조한 환자
KTAS 4단계 입니다.
응급실 가실 필요가 없습니다!
